In [1]:
from pyspark.context import SparkContext

sc = SparkContext('local', 'test')

from pyspark.sql import SQLContext

spark = SQLContext(sc)

ModuleNotFoundError: No module named 'pyspark'

In [2]:
df_train = spark.read.parquet("parquet/train")
df_test = spark.read.parquet("parquet/test")

In [5]:
!ls parquet/train

_SUCCESS
part-00000-dac68f99-685f-4651-bb02-0453bf98af03-c000.snappy.parquet


In [7]:
df_train.show(3)

+----+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+--------------+--------------+------+
| age|        workclass|  fnlwgt| education|education-num|     marital-status|        occupation|  relationship|  race|  sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+----+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-----+------------+------------+--------------+--------------+------+
|39.0|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White| Male|      2174.0|         0.0|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White| Male|         0.0|         0.0|          13.0| United-States| <=50K|
|38.0|          Private|215646.0|   HS-grad|          9

In [9]:
df_train.schema

StructType(List(StructField(age,FloatType,true),StructField(workclass,StringType,true),StructField(fnlwgt,FloatType,true),StructField(education,StringType,true),StructField(education-num,FloatType,true),StructField(marital-status,StringType,true),StructField(occupation,StringType,true),StructField(relationship,StringType,true),StructField(race,StringType,true),StructField(sex,StringType,true),StructField(capital-gain,FloatType,true),StructField(capital-loss,FloatType,true),StructField(hours-per-week,FloatType,true),StructField(native-country,StringType,true),StructField(income,StringType,true)))

In [41]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="education", outputCol="educationIndex")

indexer_model = indexer.fit(df_train)

df_indexed = indexer_model.transform(df_train)

df_indexed.toPandas()



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,educationIndex
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,2.0
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,2.0
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,0.0
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,5.0
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K,6.0
32557,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K,0.0
32558,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K,0.0
32559,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,0.0


In [45]:
indexer_model.labelsArray

[(' HS-grad',
  ' Some-college',
  ' Bachelors',
  ' Masters',
  ' Assoc-voc',
  ' 11th',
  ' Assoc-acdm',
  ' 10th',
  ' 7th-8th',
  ' Prof-school',
  ' 9th',
  ' 12th',
  ' Doctorate',
  ' 5th-6th',
  ' 1st-4th',
  ' Preschool')]

In [38]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols=["age","education-num","educationIndex"], outputCol="features") # transformer

df_vector_assembled = vector_assembler.transform(df_indexed)
df_vector_assembled.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,educationIndex,features
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,2.0,"[39.0, 13.0, 2.0]"
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,2.0,"[50.0, 13.0, 2.0]"
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,0.0,"[38.0, 9.0, 0.0]"
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,5.0,"[53.0, 7.0, 5.0]"
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,2.0,"[28.0, 13.0, 2.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K,6.0,"[27.0, 12.0, 6.0]"
32557,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K,0.0,"[40.0, 9.0, 0.0]"
32558,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K,0.0,"[58.0, 9.0, 0.0]"
32559,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,0.0,"[22.0, 9.0, 0.0]"


In [39]:
from pyspark.sql import functions as F

df_vector_and_label = df_vector_assembled.withColumn("label",F.when(F.col("Income")==' <=50K',0).otherwise(1))
df_vector_and_label.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+--------------+---------------+-----+
| age|        workclass|  fnlwgt|    education|education-num|      marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loss|hours-per-week|native-country|income|educationIndex|       features|label|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+--------------+---------------+-----+
|39.0|        State-gov| 77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|           2.0|[39.0,13.0,2.0]|    0|
|50.0| Self-emp-not-

In [40]:
from pyspark.ml.classification import NaiveBayes
naive_bayes = NaiveBayes(smoothing=1.0) # estimator 
model = naive_bayes.fit(df_vector_and_label)

In [47]:
df_indexed_test = indexer.fit(df_test).transform(df_test)

df_vector_assembled_test = vector_assembler.transform(df_indexed_test)

df_vector_and_label_test = df_vector_assembled_test.withColumn("label",F.when(F.col("Income")==' <=50K',0).otherwise(1))

df_predictions = model.transform(df_vector_and_label_test)

In [48]:
df_predictions.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,capital-loss,hours-per-week,native-country,income,educationIndex,features,label,rawPrediction,probability,prediction
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,...,0.0,40.0,United-States,<=50K,5.0,"[25.0, 7.0, 5.0]",0,"[-33.7924846759076, -35.135541711430214]","[0.7929922203087658, 0.20700777969123413]",0.0
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,0.0,50.0,United-States,<=50K,0.0,"[38.0, 9.0, 0.0]",0,"[-25.62095259297994, -26.63776271329163]","[0.7343507840278038, 0.26564921597219615]",0.0
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,...,0.0,40.0,United-States,>50K,6.0,"[28.0, 12.0, 6.0]",1,"[-45.784675242583845, -47.142578611077276]","[0.7954187288522334, 0.20458127114776656]",0.0
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,...,0.0,40.0,United-States,>50K,1.0,"[44.0, 10.0, 1.0]",1,"[-31.955987453343685, -33.01317408893335]","[0.7421525407264289, 0.2578474592735711]",0.0
4,18.0,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,...,0.0,30.0,United-States,<=50K,1.0,"[18.0, 10.0, 1.0]",0,"[-24.62553951440173, -25.725666036299607]","[0.7502838112372505, 0.24971618876274948]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39.0,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,...,0.0,36.0,United-States,<=50K,2.0,"[39.0, 13.0, 2.0]",0,"[-38.44116726609642, -39.54165489965388]","[0.750351462285497, 0.24964853771450304]",0.0
16277,64.0,?,321403.0,HS-grad,9.0,Widowed,?,Other-relative,Black,Male,...,0.0,40.0,United-States,<=50K,0.0,"[64.0, 9.0, 0.0]",0,"[-32.95140053192189, -33.92527076592537]","[0.7258902450337482, 0.27410975496625184]",0.0
16278,38.0,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,0.0,50.0,United-States,<=50K,2.0,"[38.0, 13.0, 2.0]",0,"[-38.159226960752505, -39.261366128398734]","[0.7506607065655028, 0.24933929343449723]",0.0
16279,44.0,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,...,0.0,40.0,United-States,<=50K,2.0,"[44.0, 13.0, 2.0]",0,"[-39.85086879281603, -40.9430987559296]","[0.7488014055721958, 0.2511985944278043]",0.0


In [49]:
df_predictions.createOrReplaceTempView("predictions")

In [50]:
spark.sql("select label,prediction,count(*) from predictions group by label,prediction").show()

+-----+----------+--------+
|label|prediction|count(1)|
+-----+----------+--------+
|    1|       0.0|    3846|
|    0|       0.0|   12435|
+-----+----------+--------+



In [52]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

df_labeled_train = df_train.withColumn("label",F.when(F.col("Income")==' <=50K',0).otherwise(1))

indexer_education = StringIndexer(inputCol="education", outputCol="educationIndex", handleInvalid='keep')
indexer_ocupation = StringIndexer(inputCol="occupation", outputCol="occupationIndex", handleInvalid='keep')
assembler = VectorAssembler(
    inputCols=["educationIndex", "occupationIndex", 'age'],
    outputCol="features")

naive_bayes  = NaiveBayes()

pipeline = Pipeline(stages = [indexer_education, indexer_ocupation,assembler,naive_bayes])

model = pipeline.fit(df_labeled_train)

In [53]:
df_labeled_test = df_test.withColumn("label",F.when(F.col("Income")==' <=50K',0).otherwise(1))

df_predictions = model.transform(df_labeled_test)

In [54]:
df_predictions.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,income,label,educationIndex,occupationIndex,features,rawPrediction,probability,prediction
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,...,40.0,United-States,<=50K,0,5.0,6.0,"[5.0, 6.0, 25.0]","[-32.74944548514739, -35.8359311895271]","[0.9563318387892971, 0.04366816121070303]",0.0
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,50.0,United-States,<=50K,0,0.0,10.0,"[0.0, 10.0, 38.0]","[-29.548822851159436, -34.01415162434334]","[0.9886298531344082, 0.011370146865591794]",0.0
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,...,40.0,United-States,>50K,1,6.0,12.0,"[6.0, 12.0, 28.0]","[-49.83381268820575, -55.88344035686084]","[0.9976468104865331, 0.0023531895134669004]",0.0
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,...,40.0,United-States,>50K,1,1.0,6.0,"[1.0, 6.0, 44.0]","[-24.37202684430401, -26.491596494065]","[0.8927907453174381, 0.1072092546825618]",0.0
4,18.0,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,...,30.0,United-States,<=50K,0,1.0,7.0,"[1.0, 7.0, 18.0]","[-22.202731054896386, -26.101666409380677]","[0.9801389798707989, 0.019861020129201216]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39.0,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,...,36.0,United-States,<=50K,0,2.0,0.0,"[2.0, 0.0, 39.0]","[-12.759076775986122, -12.034621361557315]","[0.3264126258509391, 0.673587374149061]",1.0
16277,64.0,?,321403.0,HS-grad,9.0,Widowed,?,Other-relative,Black,Male,...,40.0,United-States,<=50K,0,0.0,7.0,"[0.0, 7.0, 64.0]","[-27.1632693734388, -28.81659951453765]","[0.8393406207233286, 0.16065937927667137]",0.0
16278,38.0,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,50.0,United-States,<=50K,0,2.0,0.0,"[2.0, 0.0, 38.0]","[-12.588049067410285, -11.912172469944649]","[0.33718222378346835, 0.6628177762165317]",1.0
16279,44.0,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,...,40.0,United-States,<=50K,0,2.0,3.0,"[2.0, 3.0, 44.0]","[-20.44648921955769, -21.02808911135567]","[0.6414354582714246, 0.3585645417285754]",0.0
